# Adding Model Prediction to a Voxel51 Dataset
adapted from: https://voxel51.com/docs/fiftyone/recipes/model_inference.html

If you haven’t already, install FiftyOne:

In [2]:
!pip install fiftyone

     |████████████████████████████████| 25.9 MB 7.7 MB/s eta 0:00:01


  Attempting uninstall: scipy
    Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# Build FiftyOne Dataset

In [1]:
#config
import fiftyone as fo
import os
import pandas as pd


file_path = "/tf/testing"
planes = pd.read_csv("./aircraftDatabase.csv")
#print(planes)
batch_size = 1000
# Create the dataset
try:
    dataset = fo.load_dataset("plane-dataset")
except fo.core.dataset.DoesNotExistError:
    dataset = fo.Dataset(name="plane-dataset")
    


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


        icao24 registration manufacturericao               manufacturername  \
0       aa3487        N757F              NaN      Raytheon Aircraft Company   
1       ae267b         6533           VOUGHT                   Aerospatiale   
2       a4fa61        N42MH              NaN                          Piper   
3       c04101       C-FYQA              NaN  Champion Aircraft Corporation   
4       391927       F-GGJH            ROBIN                          Robin   
...        ...          ...              ...                            ...   
459994  68e5ce          NaN              NaN                            NaN   
459995  c05175       C-GEWA              NaN     Piper Aircraft Corporation   
459996  a3c671       N342TM              NaN               Levings Margaret   
459997  a94b1f       N6982U              NaN                         Mooney   
459998  ae01d6      61-2663           BOEING                         Boeing   

                       model typecode   serialnumbe

In [2]:

def buildImageList(filePath):
    labelbox_import = []
    for folder, subfolders, files in os.walk(filePath):
        for file in files:
            if file.endswith(".jpg"):
                image_filename = os.path.basename(file)
                external_id = os.path.splitext(image_filename)[0]
                image_path = os.path.abspath(os.path.join(folder, file))

                item = {"file_path": image_path,
                    "external_id": external_id}
                labelbox_import.append(item)
    return labelbox_import

def importImageList(fileList):

    samples = []
    for image in fileList:
        plane_id = image["external_id"].split("_")[0]
        plane = planes.loc[planes['icao24'] == plane_id.lower()]
        
        sample = fo.Sample(filepath=image["file_path"])
        sample["icao24"] = fo.Classification(label=plane_id.lower())
        if plane.size == 27:
            #print("Adding metadata for plane {} onto image {}".format(plane["icao24"].values[0], image["external_id"]))
            if isinstance(plane["model"].values[0], str):
                sample["model"] = fo.Classification(label=plane["model"].values[0] )
            if isinstance(plane["manufacturername"].values[0], str):
                sample["manufacturer"] = fo.Classification(label=plane["manufacturername"].values[0])
            sample.tags.append("plane")
        samples.append(sample)


    dataset.add_samples(samples)
    dataset.persistent = True
    # View summary info about the dataset
    print(dataset)

    # Print the first few samples in the dataset
    print(dataset.head())

In [3]:
image_list = buildImageList(file_path)
if len(image_list) > 0:
    print("Found {} images, processing in batches of: {}".format(len(image_list), batch_size))

    for i in range(0, len(image_list), batch_size):
        chunk = image_list[i:i + batch_size]
        importImageList(chunk)

Found 42837 images, processing in batches of: 1000
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-06-00
Adding metadata for plane a1a3a9 onto image A1A3A9_2021-01-23-20-11-31
Adding metadata for plane a19910 onto image A19910_2021-01-22-12-35-26
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-49
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-07-47
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-05-57
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-07-03
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-07-28
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-35
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-08-20
Adding metadata for plane a1b2a8 onto image A1B2A8_2021-01-22-20-41-43
Adding metadata for plane a19d0d onto image A19D0D_2021-01-20-14-29-18
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-50
Adding metadata for plane 

Adding metadata for plane a1b2a8 onto image A1B2A8_2021-01-22-20-41-12
Adding metadata for plane a19d0d onto image A19D0D_2021-01-20-14-28-18
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-08-11
Adding metadata for plane a19d0d onto image A19D0D_2021-01-20-14-29-01
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-05-49
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-06-24
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-05-11
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-00
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-05-48
Adding metadata for plane a1a3a9 onto image A1A3A9_2021-01-23-20-11-49
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-05-07
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-06-50
Adding metadata for plane a1b2a8 onto image A1B2A8_2021-01-22-20-41-49
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-07-31
Adding

Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-05-41
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-06-36
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-06-12
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-56
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-26
Adding metadata for plane a19cc7 onto image A19CC7_2021-01-20-20-21-41
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-04-50
Adding metadata for plane a19910 onto image A19910_2021-01-22-12-35-56
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-08-21
Adding metadata for plane a19d0d onto image A19D0D_2021-01-20-14-29-22
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-06-07
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-05-52
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-06-07
Adding metadata for plane a19cc7 onto image A19CC7_2021-01-20-20-21-58
Adding

Adding metadata for plane a1a3a9 onto image A1A3A9_2021-01-23-20-11-14
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-07-30
Adding metadata for plane a1b2a8 onto image A1B2A8_2021-01-22-20-41-04
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-06-17
Adding metadata for plane a1b2a8 onto image A1B2A8_2021-01-22-20-41-36
Adding metadata for plane a1a3a9 onto image A1A3A9_2021-01-23-20-10-51
Adding metadata for plane a19d0d onto image A19D0D_2021-01-20-14-29-28
Adding metadata for plane a19cc7 onto image A19CC7_2021-01-20-20-22-22
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-06-54
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-06-14
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-07-18
Adding metadata for plane a194aa onto image A194AA_2021-01-23-15-08-40
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-04-55
Adding metadata for plane a19910 onto image A19910_2021-01-22-12-35-19
Adding

Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-05-58
Adding metadata for plane a19910 onto image A19910_2021-01-22-12-35-04
Adding metadata for plane a1a3a9 onto image A1A3A9_2021-01-23-20-12-11
Adding metadata for plane a19910 onto image A19910_2021-01-22-12-35-52
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-06-09
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-55
Adding metadata for plane a194aa onto image A194AA_2021-01-22-20-07-11
Adding metadata for plane a1b2a8 onto image A1B2A8_2021-01-22-20-41-22
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-16-03
Adding metadata for plane a19cc7 onto image A19CC7_2021-01-20-20-22-05
Adding metadata for plane a19cc7 onto image A19CC7_2021-01-20-20-22-04
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-25
Adding metadata for plane a1a435 onto image A1A435_2021-01-21-21-15-09
Adding metadata for plane a1a458 onto image A1A458_2021-01-21-21-06-16
Adding

...

Adding metadata for plane abc432 onto image ABC432_2021-01-20-15-17-27
Adding metadata for plane abbd73 onto image ABBD73_2021-01-23-20-35-55
Adding metadata for plane abbd73 onto image ABBD73_2021-01-23-20-36-12
Adding metadata for plane abc432 onto image ABC432_2021-01-20-15-17-04
Adding metadata for plane abc432 onto image ABC432_2021-01-20-15-17-17
Adding metadata for plane abc432 onto image ABC432_2021-01-20-15-17-33
Adding metadata for plane a3244a onto image A3244A_2021-01-23-14-27-58
Adding metadata for plane abc432 onto image ABC432_2021-01-20-15-18-16
Adding metadata for plane a3244a onto image A3244A_2021-01-23-14-27-23
Adding metadata for plane abbd73 onto image ABBD73_2021-01-23-20-36-25
Adding metadata for plane abc432 onto image ABC432_2021-01-20-15-18-09
Adding metadata for plane abbd73 onto image ABBD73_2021-01-23-20-35-08
Adding metadata for plane a3244a onto image A3244A_2021-01-23-14-27-42
Adding metadata for plane abc432 onto image ABC432_2021-01-20-15-18-19
Adding

Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-33-51
Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-34-43
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-56-02
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-56-03
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-57-18
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-44-12
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-45-11
Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-34-27
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-43-51
Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-33-20
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-56-44
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-56-19
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-44-19
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-57-24
Adding

Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-34-14
Adding metadata for plane a455cd onto image A455CD_2021-01-23-18-06-41
Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-34-58
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-56-28
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-44-44
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-56-50
Adding metadata for plane a455cd onto image A455CD_2021-01-23-18-06-27
Adding metadata for plane a455cd onto image A455CD_2021-01-23-18-06-53
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-44-02
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-45-18
Adding metadata for plane a455cd onto image A455CD_2021-01-23-18-06-26
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-56-59
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-45-06
Adding metadata for plane a455cd onto image A455CD_2021-01-23-18-06-31
Adding

Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-34-31
Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-34-56
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-56-21
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-55-59
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-45-02
Adding metadata for plane a455cd onto image A455CD_2021-01-23-18-06-36
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-44-04
Adding metadata for plane a455cd onto image A455CD_2021-01-23-18-06-55
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-44-54
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-45-10
Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-35-01
Adding metadata for plane a48f5d onto image A48F5D_2021-01-24-14-44-23
Adding metadata for plane a6353e onto image A6353E_2021-01-21-14-34-53
Adding metadata for plane a7ef30 onto image A7EF30_2021-01-23-15-57-27
Adding

ValidationError: ValidationError (Classification:None) (StringField only accepts string values: ['label'])

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# image folder
folder_path = '/tf/testing'
# path to model
model_path = '/tf/dataset/plane-detector'
# dimensions of images
img_width, img_height = 299, 299


In [6]:
# load the trained model
model = load_model(model_path)
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)


In [7]:
print(dataset)

Name:           plane-dataset
Media type:     image
Num samples:    37000
Persistent:     True
Info:           {}
Tags:           ['plane']
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    icao24:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    model:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    manufacturer: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)


In [ ]:
#view = dataset.limit(100)
for sample in dataset: #view:
    image_size = (img_width, img_height)
    img = keras.preprocessing.image.load_img(sample.filepath, target_size=image_size)

    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis

    predictions = model.predict(img_array)
    score = predictions[0][0]
    #print(score)
    if score>0:
        label="plane"
    else:
        label="no plane"
    sample["plane"] = fo.Classification(confidence=score,label=label )
    sample.save()


In [11]:
session = fo.launch_app(dataset, auto=False)
view = dataset.exists("plane")
session.view = view

Session launched. Run `session.show()` to open the App in a cell output.


In [12]:
view = dataset.exists("plane")
session.view = view